In [7]:
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import folium
from geojson import Feature, FeatureCollection, Point
import json
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from folium import IFrame
from pyproj import Proj
from shapely.geometry import Polygon

In [8]:
# save them to csvs
bikeData = pd.read_csv("data/bikes.csv")
streets = pd.read_csv("data/streets.csv")
streetSegs = pd.read_csv("data/streetSegs.csv")

FileNotFoundError: [Errno 2] File b'data/bikes.csv' does not exist: b'data/bikes.csv'

In [ ]:
def runCluster(dataset, eps, minSamples):
    accPoints = np.empty((0, 2))   
    
    for lat,long in zip(dataset['LATITUDE'], dataset['LONGITUDE']):
        accPoints = np.append(accPoints, np.array([[lat,long]]), axis=0)

    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=minSamples).fit(accPoints)

    clusters = db.fit_predict(accPoints)
    dataset['CLUSTER'] = clusters
    

    clusters = dataset['CLUSTER'].unique()   
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
    primaryFacilityTypeDict = {}
    secondaryFacilityTypeDict = {}
    primaryStreetSegIdDict = {}
    secondaryStreetSegIdDict = {}
    
    for i in clusters:
        clusterSet = dataset[(dataset["CLUSTER"] == i) & (dataset["STREETSEGID"] > 0)]
        valueCounts = clusterSet['STREETSEGID'].value_counts()
        if (len(valueCounts.index) > 1):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["STREETSEGID"] == int(primaryValueId)].reset_index()["REGISTEREDNAME"][0]
            secondaryValueId = valueCounts.index[1]
            secondaryValue = streetSegs[streetSegs["STREETSEGID"] == int(secondaryValueId)].reset_index()["REGISTEREDNAME"][0]
            primaryStreetDict[i] = primaryValue
            secondaryStreetDict[i] = secondaryValue
            
            facilityResp = streets[streets["STREETSEGID"] == int(primaryValueId)]
            if (len(facilityResp) > 0):    
                primaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
                primaryStreetSegIdDict[i] = primaryValueId
            else:
                primaryFacilityTypeDict[i] = "None"
                primaryStreetSegIdDict[i] = "None"
            facilityResp = streets[streets["STREETSEGID"] == int(secondaryValueId)]
            if (len(facilityResp) > 0):  
                secondaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
                secondaryStreetSegIdDict[i] = secondaryValueId
            else:
                secondaryFacilityTypeDict[i] = "None"
                secondaryStreetSegIdDict[i] = "None"
        elif (len(valueCounts.index) > 0):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["STREETSEGID"] == int(primaryValueId)].reset_index()["REGISTEREDNAME"][0]
            primaryStreetDict[i] = primaryValue
            facilityResp = streets[streets["STREETSEGID"] == int(primaryValueId)]
            if (len(facilityResp) > 0):    
                primaryFacilityTypeDict[i] = facilityResp.reset_index()["FACILITY"][0]
                primaryStreetSegIdDict[i] = primaryValueId
            else:
                primaryFacilityTypeDict[i] = "None"
                primaryStreetSegIdDict[i] = "None"
                
            secondaryStreetDict[i] = "None"
            secondaryFacilityTypeDict[i] = "None"
            secondaryStreetSegIdDict[i] = "None"
        else:
            primaryStreetDict[i] = "None"
            secondaryStreetDict[i] = "None"
            primaryFacilityTypeDict[i] = "None"
            secondaryFacilityTypeDict[i] = "None"
            primaryStreetSegIdDict[i] = "None"
            secondaryStreetSegIdDict[i] = "None"
            
    primaryStreets = []
    secondaryStreets = []
    primaryFacilityTypes = []
    secondaryFacilityTypes = []
    primaryStreetSegIds = []
    secondaryStreetSegIds = []

    for i in dataset["CLUSTER"]:
        if (primaryStreetDict[i] == "None"):
            primaryStreets.append("None")
            primaryFacilityTypes.append("None")
            primaryStreetSegIds.append("None")
        else:
            primaryStreets.append(primaryStreetDict[i])
            primaryFacilityTypes.append(primaryFacilityTypeDict[i])
            primaryStreetSegIds.append(primaryStreetSegIdDict[i])
        if (secondaryStreetDict[i] == "None"):
            secondaryStreets.append("None")
            secondaryFacilityTypes.append("None")
            secondaryStreetSegIds.append("None")
        else:
            secondaryStreets.append(secondaryStreetDict[i])
            secondaryFacilityTypes.append(secondaryFacilityTypeDict[i])
            secondaryStreetSegIds.append(secondaryStreetSegIdDict[i])
       
    dataset['CLUSTERPRIMARYSTREET'] = primaryStreets
    dataset['CLUSTERSECONDARYSTREET'] = secondaryStreets
    dataset['CLUSTERprimaryFACILITYTYPE'] = primaryFacilityTypes
    dataset['CLUSTERSECONDARYFACILITYTYPE'] = secondaryFacilityTypes
    dataset['CLUSTERPRIMARYSTREETSEGID'] = primaryStreetSegIds
    dataset['CLUSTERSECONDARYSTREETSEGID'] = secondaryStreetSegIds
    return dataset

In [ ]:
clusteredDataSmall = runCluster(bikeData, 0.0015, 25)

In [ ]:
clusteredDataMedium = runCluster(bikeData, 0.0014, 15)

In [ ]:
clusteredDataLarge = runCluster(bikeData, 0.0005, 7)